<a href="https://colab.research.google.com/github/a-afonso/03MAIR-Algoritmos-de-optimizacion/blob/master/SEMINARIO/SEMINARIO_Augusto_Afonso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
<b>Nombre y Apellidos: </b>Augusto Afonso Losada <br>
<b>Url: </b>https://github.com/a-afonso/03MAIR-Algoritmos-de-optimizacion/tree/master/SEMINARIO<br>
  <b>Problema:</b>
> 1. Elección de grupos de población homogéneos <br>

<b>Descripción del problema: </b>Una productora ganadera nos encarga la tarea de seleccionar grupos de terneros para aplicar 3 tratamientos diferentes. Para cada uno de los tratamientos debemos seleccionar 3 grupos de terneros que sean lo mas homogéneos posible en peso para que en los resultados del tratamiento influya lo menos posible el peso del animal. Disponemos de una población de N animales entre machos y hembras.<br>
<br>
Se solicita diseñar un algoritmo para conseguir una agrupación que cumpla de la mejor manera posible las especificaciones de la productora.<br>
<br>

(*) La respuesta es obligatoria





                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?

¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?





**Respuesta**<br>

Para calcular las posibilidades sin restricciones, debemos suponer que son N animales (independientemente del sexo) y que el peso no influye en la solución.<br>

En estas circunstancias, tenemos 3 tratamientos x 3 grupos = 9 grupos efectivos, por lo cual lo que tenemos son combinaciones de N elementos tomados de 9 en 9 y sin repetición, es decir:<br>

*   N! / 9! (N-9)!

<br>
En el caso con restricciones, aunque podría parecer que la solución puede ser única (la que proponga la combinación más homogénea), en realidad no es posible determinar con exactitud el número de posibles soluciones, ya que por ejemplo, puede haber animales del mismo sexo con el mismo peso, en cuyo caso podrían caer en cualquiera de los grupos efectivos del propio sexo indistintamente.<br>

La peor de las situaciones sería aquella en la que todos los animales tienen el mismo peso y el sexo está divido exactamente al 50%, porque cualquier combinación sería posible, con lo cual las posibilidades coincidirían con el caso anterior:<br>

*   N! / 9! (N-9)!

Paradójicamente, en este último caso cualquier combinación sería la solución óptima, ya que cumpliría con el criterio solicitado.<br>

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Arguméntalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo).


**Respuesta**

Es posible utilizar una lista de listas para contener la estructura de datos de entrada. La lista contendría N elementos de tipo lista. En esta última estructura, habría tres valores: el identificador del animal, el peso y el sexo.<br>

La estructura de datos devuelta sería una lista de 9 elementos, tantos como grupos efectivos de cada sexo, que a su vez serían listas. Estas últimas tendrían una estructura en la cual sus elementos serían los identificadores de los animales de cada uno de los grupos efectivos.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

**Respuesta**

Para responder a estas preguntas, habría que definir el concepto de *"Homogeneidad de un Grupo Efectivo de animales"*.<br>

*   Homogeneidad de un Grupo Efectivo de animales (HGE): Peso medio de los animales que componen un grupo efectivo de animales. Donde un grupo efectivo de animales (GE) equivale al conjunto de animales en cada uno de los conjuntos formados por el cómputo de 3 tratamientos x 3 grupos. Es decir hay 9 grupos efectivos (por cada sexo) con sus animales.

Para un determinado GE, habría que calcular su HGE, comparándola con lo desviada que está respecto al peso medio de toda la muestra. La combinación óptima será aquella en la que dicha desviación de todos los GE con respecto al peso medio, es mínima.<br>

Por lo tanto es un problema de minimización. 



Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [24]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from itertools import permutations

# Inicialización de variables globales
#   - Escogemos solo una población de 18 animales y 3 grupos por sexo (1x3) 
#     para las pruebas porque la explosión combinatoria hace que el tiempo
#     de ejecución sea excesivo a partir de estos valores
#   - Incluso en este caso, si el número de animales queda dividido por sexo
#     al 50% (9+9), el tiempo se hace inviable
nIndividuals = 18
nTreats = 1 
nGroups = 3

# Creación de un dataset de muestra
MUESTRA = pd.DataFrame()
MUESTRA["Id"] = [i for i in range(nIndividuals)]
MUESTRA["Peso"] = [random.gauss(37, 2.1) for i in range(nIndividuals)]
MUESTRA["Sexo"] = [random.getrandbits(1) for i in range(nIndividuals)]

# Función que resuelve el problema 1 del seminario
def do_problem1(MUESTRA, nTreats, nGroups):
  nAnimals = len(MUESTRA)

  # Dividimos el dataset en dos listas, una para cada sexo
  MALE = []
  FEMALE = []

  for index, row in MUESTRA.iterrows():
    if row["Sexo"] == 0:
      MALE.append([int(row["Id"]), row["Peso"], 0])
    else:
      FEMALE.append([int(row["Id"]), row["Peso"], 0])

  # Calcula el número máximo de animales por grupo
  nMale = len(MALE)
  nFemale = len(FEMALE)

  nAnimalsGroup = min(nMale, nFemale) // (nTreats * nGroups)

  # Función local que calcula la homogeneidad de los pesos de una 
  # permutación
  def get_homogeneidad(PERMUTACION, PESOS, p_medio, nTotalGroups, nAnimalsGroup):
    TEMP_1 = []
    ix = 0
    suma = 0
    for g in range(nTotalGroups):
      TEMP_2 = []
      TEMP_3 = []
      for i in range(ix, ix+nAnimalsGroup):
        TEMP_2.append(PESOS[i])
        TEMP_3.append(PERMUTACION[i])
      TEMP_1.append(TEMP_3)
      ix = ix + nAnimalsGroup

      media = np.mean(TEMP_2)
      suma = suma + abs(p_medio - media)

    #print(TEMP_1)

    return(suma / nTotalGroups, TEMP_1)

  # Función local para crear agrupaciones de cualquier sexo
  def group_animals(MSEXO, nTreats, nGroups, nAnimalsGroup):

    # Calcula el peso medio
    p_medio = np.mean(MSEXO, axis=0)[1]
    #print(p_medio)
    
    # Calcula el número total de grupos
    nTotalGroups = nTreats * nGroups

    # Calcula el número de animales de la lista
    nAnimalsSex = nTotalGroups * nAnimalsGroup

    # Ordenamos la muestra por diferencias con respecto a la media
    # para excluir los más distantes en peso (de la media)
    #print(MSEXO)
    for a in MSEXO:
      a[2] = abs(p_medio - a[1])
    MSEXO = sorted(MSEXO, key=lambda x: x[2])
    #print(MSEXO)
    MSEXO = MSEXO[0:nAnimalsSex]
    #print(MSEXO)

    # Calcula el nuevo peso medio
    p_medio = np.mean(MSEXO, axis=0)[1]
    #print(p_medio)

    # Calcula las permutaciones de animales y crea una lista equivalente
    # a ellas con sus pesos correspondientes
    PERMUTACIONES = list(permutations([MSEXO[i][0] for i in range(0, len(MSEXO))]))
    #print(PERMUTACIONES)
    #input("Press Enter to continue...")
    PESOS = []
    
    for PERM in PERMUTACIONES:
      P = []
      for p in PERM:
        P.append(MUESTRA.loc[MUESTRA["Id"] == p, "Peso"].iloc[0])
      PESOS.append(P)

    #print(PESOS) 
    #input("Press Enter to continue...")

    # Calcula la permutación más homogénea
    best_permutacion = []
    best_homogeneidad = 99999999
    for ix in range(len(PERMUTACIONES)):
      h, g = get_homogeneidad(PERMUTACIONES[ix], PESOS[ix], p_medio, nTotalGroups, nAnimalsGroup)
      if h < best_homogeneidad:
        best_permutacion = g
        best_homogeneidad = h

    return best_permutacion, h
  
  # Consigue las agrupaciones de animales macho más homogéneas
  #print('Sexo M')
  RMALE, hM = group_animals(MALE, nTreats, nGroups, nAnimalsGroup)
  #print('Sexo H')
  # Consigue las agrupaciones de animales hembra más homogéneas
  RFEMALE, hF = group_animals(FEMALE, nTreats, nGroups, nAnimalsGroup)
  
  return RMALE, RFEMALE


# Calcula e imprime los grupos para una muestra dada
agrupaciones = do_problem1(MUESTRA, nTreats, nGroups)

print('Grupos de sexo Macho: ', agrupaciones[0])
print()
print('Grupos de sexo Hembra: ', agrupaciones[1])


Grupos de sexo Macho:  [[17, 10], [12, 7], [14, 4]]

Grupos de sexo Hembra:  [[5, 1], [15, 0], [13, 9]]


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [26]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from itertools import permutations

# Inicialización de variables globales
nIndividuals = 300
nTreats = 3 
nGroups = 3

# Creación de un dataset de muestra
MUESTRA = pd.DataFrame()
MUESTRA["Id"] = [i for i in range(nIndividuals)]
MUESTRA["Peso"] = [random.gauss(37, 2.1) for i in range(nIndividuals)]
MUESTRA["Sexo"] = [random.getrandbits(1) for i in range(nIndividuals)]

# Función mejorada que resuelve el problema 1 del seminario
def do_problem1(MUESTRA, nTreats, nGroups):
  nAnimals = len(MUESTRA)

  # Dividimos el dataset en dos listas, una para cada sexo
  MALE = []
  FEMALE = []

  for index, row in MUESTRA.iterrows():
    if row["Sexo"] == 0:
      MALE.append([int(row["Id"]), row["Peso"], 0])
    else:
      FEMALE.append([int(row["Id"]), row["Peso"], 0])

  # Calcula el número máximo de animales por grupo
  nMale = len(MALE)
  nFemale = len(FEMALE)

  nAnimalsGroup = min(nMale, nFemale) // (nTreats * nGroups)

  # Función local para crear agrupaciones de cualquier sexo
  def group_animals(MSEXO, nTreats, nGroups, nAnimalsGroup):

    # Calcula el peso medio
    p_medio = np.mean(MSEXO, axis=0)[1]
    
    # Calcula el número total de grupos
    nTotalGroups = nTreats * nGroups

    # Calcula el número de animales de la lista
    nAnimalsSex = nTotalGroups * nAnimalsGroup

    # Ordenamos la muestra por diferencias con respecto a la media
    # para excluir los más distantes en peso (de la media)
    for a in MSEXO:
      a[2] = abs(p_medio - a[1])
    MSEXO = sorted(MSEXO, key=lambda x: x[2])
    MSEXO = MSEXO[0:nAnimalsSex]
    
    # Reordenamos la muestra por pesos
    MSEXO = sorted(MSEXO, key=lambda x: x[1])

    # Calcula el nuevo peso medio
    #p_medio = np.mean(MSEXO, axis=0)[1]

    # Crea los grupos más homogéneos posible, escogiendo un individuo por cada
    # nTotalGroups, de forma que en cada grupo entren individuos significativos 
    # de cada zona de la lista ya ordenada por pesos
    TEMP_1 = []
    for i in range(nTotalGroups):
      TEMP_2 = []
      
      for j in range(i, nAnimalsSex, nTotalGroups):
        TEMP_2.append(MSEXO[j])
      
      TEMP_1.append(TEMP_2)
    
    return TEMP_1
  
  # Consigue las agrupaciones de animales macho más homogéneas
  RMALE = group_animals(MALE, nTreats, nGroups, nAnimalsGroup)
  
  # Consigue las agrupaciones de animales hembra más homogéneas
  RFEMALE = group_animals(FEMALE, nTreats, nGroups, nAnimalsGroup)
  
  return RMALE, RFEMALE


# Calcula e imprime los grupos para una muestra dada
agrupaciones = do_problem1(MUESTRA, nTreats, nGroups)

print('Grupos de sexo Macho: ')
for i in agrupaciones[0]:
  print(i)
print()

print('Grupos de sexo Hembra: ')
for i in agrupaciones[1]:
  print(i)


Grupos de sexo Macho: 
[[114, 32.54611166502898, 4.242481849240065], [206, 33.96326436326299, 2.825329151006059], [227, 34.6041129658702, 2.184480548398845], [98, 34.913972097985464, 1.8746214162835813], [90, 35.26791024050469, 1.5206832737643552], [10, 35.75096620758292, 1.037627306686126], [125, 35.91952219436217, 0.8690713199068725], [91, 36.230114762605744, 0.5584787516633014], [152, 36.57963756440449, 0.2089559498645528], [11, 36.94515980907206, 0.15656629480301376], [3, 37.51454544106804, 0.7259519267989916], [115, 37.80081762320457, 1.012224108935527], [69, 38.32900591697148, 1.5404124027024366], [46, 38.60758902244883, 1.818995508179782], [29, 38.83900061766573, 2.0504071033966866], [270, 39.58183865485216, 2.7932451405831173]]
[[264, 32.643776250912836, 4.1448172633562095], [1, 33.97046923800065, 2.8181242762683922], [296, 34.667398117150015, 2.121195397119031], [88, 35.052139915253704, 1.7364535990153414], [43, 35.27713845834292, 1.5114550559261275], [229, 35.769982558854686,

(*)Calcula la complejidad del algoritmo 

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [0]:
import pandas as pd
import random

# Inicialización de variables globales
nAnimals = 3000
nTreats = 3
nGroups = 3

# Creación de un dataset de muestra
MUESTRA = pd.DataFrame()
MUESTRA["Id"] = [i for i in range(nAnimals)]
MUESTRA["Peso"] = [random.gauss(37, 2.1) for i in range(nAnimals)]
MUESTRA["Sexo"] = [random.getrandbits(1) for i in range(nAnimals)]

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta